In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import CamembertTokenizer, CamembertForSequenceClassification, Trainer, TrainingArguments
import torch

# Étape 1 : Chargement des données
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Étape 2 : Prétraitement des données
# Supposons que vos données contiennent deux colonnes : 'text' (titre + instructions) et 'label' (Plat principal, Entrée, Dessert)
# Vous devez d'abord convertir les étiquettes en indices numériques
label_dict = {"Plat principal": 0, "Entrée": 1, "Dessert": 2}
train_data['type'] = train_data['type'].replace(label_dict)
test_data['type'] = test_data['type'].replace(label_dict)

# Tokenisation
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")

def tokenize_function(examples):
    return tokenizer(examples['titre'], padding="max_length", truncation=True)

train_encodings = tokenize_function(train_data.to_dict(orient='list'))
test_encodings = tokenize_function(test_data.to_dict(orient='list'))

# Convertir en Dataset de PyTorch
class RecipesDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RecipesDataset(train_encodings, train_data['type'].tolist())
test_dataset = RecipesDataset(test_encodings, test_data['type'].tolist())

# Étape 3 : Entraînement du modèle
model = CamembertForSequenceClassification.from_pretrained("camembert-base", num_labels=3)

training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,   
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
)

trainer.train()

# Étape 4 : Évaluation du modèle
def get_predictions(model, dataset):
    outputs = trainer.predict(test_dataset)
    predictions = outputs.predictions.argmax(-1)
    return predictions

predictions = get_predictions(model, test_dataset)
accuracy = accuracy_score(test_data['type'].tolist(), predictions)
print(f'Accuracy: {accuracy}')


/var/folders/y5/8dfcz8r14gl9d6fvjl93ctn80000gn/T/ipykernel_1730/4097286818.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['type'] = train_data['type'].replace(label_dict)
/var/folders/y5/8dfcz8r14gl9d6fvjl93ctn80000gn/T/ipykernel_1730/4097286818.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['type'] = test_data['type'].replace(label_dict)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassifi